In [87]:
from astropy.io import fits
import numpy as np
from astropy.table import Table
from decimal import *
import pandas as pd
from math import *
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import ascii
import glob
import os
from pathlib import Path

In [88]:
sb_es1_bcgs = ["ES1_z_0.88", "ES1_z_0.99", "ES1_z_0.99b", "ES1_z_1.04", "ES1_z_1.38", "ES1_z_1.40", "ES1_z_1.60", "ES1_z_1.65", "ES1_z_1.70"]
sb_xmm_bcgs = ["XMM_113", "XMM_z_0.9", "XMM_z_1.0", "XMM_z_0.81"]

In [89]:
sb_bcgs = sb_es1_bcgs + sb_xmm_bcgs
# for bcg in sb_bcgs:
#     Path(bcg).mkdir(parents = True, exist_ok = True)

In [90]:
bcg_coords = pd.read_csv("Starburst_centers.txt", delim_whitespace = True, names = ['id', 'ra', 'dec']).sort_values('id', ascending = True)

DES flux catalog

In [171]:
des_catalog_fluxes = pd.DataFrame(columns = ['des_g', 'des_r', 'des_i', 'des_z', 'des_Y', 'des_g_err','des_r_err', 'des_i_err', 'des_z_err', 'des_Y_err', 'id'])
for bcg in sb_bcgs:
    des_df = pd.read_csv(bcg + "/des_catalog.csv")
    cols = ['ra', 'dec', 'mag_auto_g', 'mag_auto_r', 'mag_auto_i', 'mag_auto_z','mag_auto_y','magerr_auto_g', 'magerr_auto_r', 'magerr_auto_i','magerr_auto_z', 'magerr_auto_y']
    des_fluxes = pd.DataFrame(columns = cols)
    def mag_to_mJy(mag):
        return 10**((8.90 - mag)/2.5)*1000
    mag_cols = ['mag_auto_g', 'mag_auto_r', 'mag_auto_i', 'mag_auto_z','mag_auto_y']
    magerr_cols = ['magerr_auto_g', 'magerr_auto_r', 'magerr_auto_i','magerr_auto_z', 'magerr_auto_y']
    ra = bcg_coords[bcg_coords['id'] == bcg]['ra'].values[0]
    dec = bcg_coords[bcg_coords['id'] == bcg]['dec'].values[0]
    c = SkyCoord(ra = ra*u.degree, dec = dec*u.degree, frame = 'icrs')
    des_catalog = SkyCoord(ra=np.array(des_df['ra'])*u.degree, dec=np.array(des_df['dec'])*u.degree, frame = 'icrs')
    idx_all = c.separation(des_catalog) < 0.004*u.deg
    idx, d2d, d3d = c.match_to_catalog_sky(des_catalog)
    match = pd.DataFrame(des_df.iloc[idx]).T.reset_index(drop = True)
    nearest_points = pd.DataFrame(des_df.iloc[idx_all]).reset_index(drop = True)
    match_jy = match[['ra', 'dec']].copy()
    for err_col,col in zip(magerr_cols, mag_cols):
        match_jy[col] = mag_to_mJy(match[col])
        match_jy[err_col] = match[err_col]*match_jy[col]
    match_jy['id'] = bcg
    des_fluxes = pd.concat([des_fluxes, match_jy])
    des_fluxes = des_fluxes.rename(columns = {"mag_auto_g":"des_g", "magerr_auto_g": "des_g_err", "mag_auto_r": "des_r", "magerr_auto_r": "des_r_err",
                     "mag_auto_i": "des_i", "magerr_auto_i":"des_i_err", "mag_auto_z":"des_z", "magerr_auto_z":"des_z_err",
                     "mag_auto_y":"des_Y", "magerr_auto_y": "des_Y_err"})
    with open(bcg + "/" + bcg+'.reg','w') as f:
        region = "# Region file format: DS9 version 4.0 \nglobal color=blue font='helvetica 10 normal roman' edit=1 move=1 delete=1 highlite=1 include=1 wcs=wcs \nfk5; circle " + str(des_fluxes['ra'].values[0]) + " " + str(des_fluxes['dec'].values[0]) + " 0.5" + '"'
        f.write(region)
    header = "# Region file format: DS9 version 4.0 \nglobal color=red font='helvetica 10 normal roman' edit=1 move=1 delete=1 highlite=1 include=1 wcs=wcs"
    f = open(bcg + "/" + bcg+'_all.reg','w')
    f.write(header)
    for ind,row in nearest_points.iterrows():
        region = "\nfk5; cross point " + str(row['ra']) + " " + str(row['dec'])
        f.write(region)
    des_flux = des_fluxes[['des_g', 'des_r', 'des_i', 'des_z', 'des_Y', 'des_g_err','des_r_err', 'des_i_err', 'des_z_err', 'des_Y_err', 'id']].copy()
    des_catalog_fluxes = des_catalog_fluxes.append(des_flux, ignore_index = True)

In [58]:
irac_es1_coord = pd.read_csv("sb_es1_irac.csv", usecols = ['ra_01', 'dec_01', 'ra', 'dec'])
irac_es1_fluxes = pd.read_csv("sb_es1_irac.csv", usecols = ['flux_kr_36', 'uncf_kr_36','flux_kr_45', 'uncf_kr_45', 'flux_kr_58','uncf_kr_58', 'flux_kr_80', 'uncf_kr_80']).fillna(0)
irac_es1_fluxes = irac_es1_fluxes * 0.001
irac_es1_catalog = pd.concat([irac_es1_coord, irac_es1_fluxes], axis = 1)
irac_es1_catalog = irac_es1_catalog.rename(columns = {"flux_kr_36":"spitzer.irac.ch1", "uncf_kr_36": "spitzer.irac.ch1_err","flux_kr_45": "spitzer.irac.ch2", 'uncf_kr_45': "spitzer.irac.ch2_err", 'flux_kr_58': "spitzer.irac.ch3", 'uncf_kr_58': "spitzer.irac.ch3_err", 'flux_kr_80': "spitzer.irac.ch4", 'uncf_kr_80': "spitzer.irac.ch4_err"})

In [59]:
irac_xmm_coord = pd.read_csv("sb_xmm_irac.csv", usecols = ['ra_01', 'dec_01', 'ra', 'dec'])
irac_xmm_fluxes = pd.read_csv("sb_xmm_irac.csv", usecols = ['flux_kr_36', 'uncf_kr_36','flux_kr_45', 'uncf_kr_45', 'flux_kr_58','uncf_kr_58', 'flux_kr_80', 'uncf_kr_80']).fillna(0)
irac_xmm_fluxes = irac_xmm_fluxes * 0.001
irac_xmm_catalog = pd.concat([irac_xmm_coord, irac_xmm_fluxes], axis = 1)
irac_xmm_catalog = irac_xmm_catalog.rename(columns = {"flux_kr_36":"spitzer.irac.ch1", "uncf_kr_36": "spitzer.irac.ch1_err","flux_kr_45": "spitzer.irac.ch2", 'uncf_kr_45': "spitzer.irac.ch2_err", 'flux_kr_58': "spitzer.irac.ch3", 'uncf_kr_58': "spitzer.irac.ch3_err", 'flux_kr_80': "spitzer.irac.ch4", 'uncf_kr_80': "spitzer.irac.ch4_err"})

In [60]:
sb_irac_catalog = pd.concat([irac_es1_catalog, irac_xmm_catalog], axis = 0).reset_index(drop = True)
sb_irac_catalog = sb_irac_catalog.rename(columns = {'ra' : 'ra_bcg', 'dec': 'dec_bcg', 'ra_01': 'ra', 'dec_01': 'dec'})
bcgs = []
for idx,row in sb_irac_catalog.iterrows():
    bcg = bcg_coords[bcg_coords['ra'] == row['ra']]['id'].values[0]
    bcgs.append(bcg)
sb_irac_catalog['id'] = bcgs
for bcg in sb_bcgs:
    ra = sb_irac_catalog[sb_irac_catalog['id'] == bcg]['ra_bcg'].values[0]
    dec = sb_irac_catalog[sb_irac_catalog['id'] == bcg]['dec_bcg'].values[0]
    with open(bcg + "/" + bcg + "_irac.reg","w") as f:
        region = "# Region file format: DS9 version 4.0 \nglobal color=red font='helvetica 10 normal roman' edit=1 move=1 delete=1 highlite=1 include=1 wcs=wcs \nfk5; cross point " + str(ra) + " " + str(dec)
        f.write(region)
sb_irac = sb_irac_catalog[['spitzer.irac.ch1',
       'spitzer.irac.ch1_err', 'spitzer.irac.ch2', 'spitzer.irac.ch2_err',
       'spitzer.irac.ch3', 'spitzer.irac.ch3_err', 'spitzer.irac.ch4',
       'spitzer.irac.ch4_err', 'id']].copy()

In [61]:
irac_des_fluxes = pd.merge(des_catalog_fluxes, sb_irac, on='id')

In [62]:
id_es1 = bcg_coords[bcg_coords['id'].str.contains("ES1")]
id_xmm = bcg_coords[bcg_coords['id'].str.contains("XMM")]
# id_cdfs = irac_des_fluxes[irac_des_fluxes['id'].str.contains("CDFS")]
xmm_f = fits.open("L6-XMM-LSS-SWIRE_xID24_DR3.fits")
es1_f = fits.open("L6-ELAIS-S1-SWIRE_xID24_DR3.fits")
# cdfs_f = fits.open("L5-CDFS-SWIRE_xID24_DR3.fits")
es1_hermes = Table(es1_f[1].data).to_pandas()
xmm_hermes = Table(xmm_f[1].data).to_pandas()
# cdfs_hermes = Table(cdfs_f[1].data).to_pandas()
es1_h_catalog = SkyCoord(ra=np.array(es1_hermes['RA'])*u.degree, dec=np.array(es1_hermes['Dec'])*u.degree, frame = 'icrs')
xmm_h_catalog = SkyCoord(ra=np.array(xmm_hermes['RA'])*u.degree, dec=np.array(xmm_hermes['Dec'])*u.degree, frame = 'icrs')
# cdfs_h_catalog = SkyCoord(ra=np.array(cdfs_hermes['RA'])*u.degree, dec=np.array(cdfs_hermes['Dec'])*u.degree, frame = 'icrs')

In [63]:
es1_hermes_catalog = pd.DataFrame()
for ind,row in id_es1.iterrows():
    c = SkyCoord(ra = row['ra']*u.degree, dec = row['dec']*u.degree, frame = 'icrs')
    idx, d2d, d3d = c.match_to_catalog_sky(es1_h_catalog)
    es1_h_match =  pd.DataFrame(es1_hermes.iloc[idx]).T.reset_index(drop = True)
    es1_h_match['id'] = row['id']
    es1_hermes_catalog = pd.concat([es1_hermes_catalog, es1_h_match])
xmm_hermes_catalog = pd.DataFrame()
for ind,row in id_xmm.iterrows():
    c = SkyCoord(ra = row['ra']*u.degree, dec = row['dec']*u.degree, frame = 'icrs')
    idx, d2d, d3d = c.match_to_catalog_sky(xmm_h_catalog)
    xmm_h_match =  pd.DataFrame(xmm_hermes.iloc[idx]).T.reset_index(drop = True)
    xmm_h_match['id'] = row['id']
    xmm_hermes_catalog = pd.concat([xmm_hermes_catalog, xmm_h_match])

In [64]:
hermes_reg = xmm_hermes_catalog[['RA','Dec','F24', 'e_F24', 'F250', 'et_F250', 'F350', 'et_F350', 'F500', 'et_F500', 'id']]
for ind,row in hermes_reg.iterrows():
    with open(row['id'] + "/" + row['id'] + "_hermes.reg","w") as f:
        region = "# Region file format: DS9 version 4.0 \nglobal color=red font='helvetica 10 normal roman' edit=1 move=1 delete=1 highlite=1 include=1 wcs=wcs \nfk5; cross point " + str(row['RA']) + " " + str(row['Dec'])
        f.write(region)

In [65]:
es1_hermes_catalog["F24"] = es1_hermes_catalog["F24"]*0.001
es1_hermes_catalog["e_F24"] = es1_hermes_catalog["e_F24"]*0.001
xmm_hermes_catalog["F24"] = xmm_hermes_catalog["F24"]*0.001
xmm_hermes_catalog["e_F24"] = xmm_hermes_catalog["e_F24"]*0.001

In [66]:
es1_hermes_catalog = es1_hermes_catalog.reset_index(drop=True)
xmm_hermes_catalog = xmm_hermes_catalog.reset_index(drop=True)

In [67]:
hermes = pd.concat([es1_hermes_catalog, xmm_hermes_catalog]).reset_index(drop = True)
all_fluxes = pd.merge(irac_des_fluxes, hermes[['F24', 'e_F24', 'F250', 'et_F250', 'F350', 'et_F350', 'F500', 'et_F500', 'id']], on = 'id')
all_fluxes = all_fluxes.rename(columns = {"F24":"spitzer.mips.24", "e_F24": "spitzer.mips.24_err", "F250": "herschel.spire.PSW", "et_F250": "herschel.spire.PSW_err","F350": "herschel.spire.PMW", "et_F350":"herschel.spire.PMW_err", "F500": "herschel.spire.PLW", "et_F500":"herschel.spire.PLW_err"})

In [86]:
np.array(all_fluxes['id'])

array(['ES1_z_0.88', 'ES1_z_0.99', 'ES1_z_0.99b', 'ES1_z_1.04',
       'ES1_z_1.38', 'ES1_z_1.40', 'ES1_z_1.60', 'ES1_z_1.65',
       'ES1_z_1.70', 'XMM_113', 'XMM_z_0.9', 'XMM_z_1.0', 'XMM_z_0.81'],
      dtype=object)

In [69]:
all_fluxes['redshift'] = [0.56395,1.19372,0.91886,1.04,1.38,1.40,1.60,1.65,0.91651,1.6, 0.84957,0.5352355,0.7827207]

In [70]:
final_flux_table = all_fluxes[['id','redshift','des_g', 'des_r', 'des_i', 'des_z', 'des_Y', 'des_g_err',
       'des_r_err', 'des_i_err', 'des_z_err', 'des_Y_err',
       'spitzer.irac.ch1', 'spitzer.irac.ch1_err', 'spitzer.irac.ch2',
       'spitzer.irac.ch2_err', 'spitzer.irac.ch3', 'spitzer.irac.ch3_err',
       'spitzer.irac.ch4', 'spitzer.irac.ch4_err', 'spitzer.mips.24',
       'spitzer.mips.24_err', 'herschel.spire.PSW', 'herschel.spire.PSW_err',
       'herschel.spire.PMW', 'herschel.spire.PMW_err', 'herschel.spire.PLW',
       'herschel.spire.PLW_err']]

In [71]:
alma_filters = ["ES1_z_0.88", "ES1_z_0.99", "ES1_z_0.99b", "ES1_z_1.04", "XMM_113", "XMM_z_0.9", "XMM_z_1.0", "XMM_z_0.81"]
alma_filters_highz = ["ES1_z_1.38", "ES1_z_1.40", "ES1_z_1.60", "ES1_z_1.65", "ES1_z_1.70"]

In [72]:
alma_highz_flux = np.array([515, 465, 290, 1980, 971])*0.001
alma_highz_flux_unc = np.array([57, 58, 38, 110, 67])*0.001
sb_alma_flux = np.array([106, 357, 288, 470, 354, 668, 253, 397])*0.001
sb_alma_flux_unc = np.array([30, 78, 81, 84, 56, 147, 48, 86])*0.001

In [82]:
pd.set_option('display.max_columns', None)
for bcg,flux,err in zip(alma_filters, sb_alma_flux, sb_alma_flux_unc):
    ind_flux = final_flux_table[final_flux_table['id'] == bcg].copy()
    ind_flux[bcg] = [flux]
    ind_flux[bcg+'_err'] = [err]
    print(ind_flux)
#     ind_flux.to_csv(bcg+"/flux.csv", index = None)

           id  redshift    des_g    des_r     des_i     des_z     des_Y  \
0  ES1_z_0.88   0.56395  0.00414  0.01554  0.026555  0.036578  0.041784   

   des_g_err  des_r_err  des_i_err  des_z_err  des_Y_err  spitzer.irac.ch1  \
0   0.000145     0.0002   0.000345   0.000642   0.001631           0.10041   

   spitzer.irac.ch1_err  spitzer.irac.ch2  spitzer.irac.ch2_err  \
0                0.0014           0.07239               0.00172   

   spitzer.irac.ch3  spitzer.irac.ch3_err  spitzer.irac.ch4  \
0           0.08025               0.00665           0.07769   

   spitzer.irac.ch4_err spitzer.mips.24 spitzer.mips.24_err  \
0               0.00664         0.65296             0.01968   

  herschel.spire.PSW herschel.spire.PSW_err herschel.spire.PMW  \
0           30.12666                4.87164           7.174814   

  herschel.spire.PMW_err herschel.spire.PLW herschel.spire.PLW_err  \
0               7.983726           6.815913               5.413804   

   ES1_z_0.88  ES1_z_0.88_err

In [83]:
for bcg,flux,err in zip(alma_filters_highz, alma_highz_flux, alma_highz_flux_unc):
    ind_flux = final_flux_table[final_flux_table['id'] == bcg].copy()
    ind_flux["es1_highz_alma"] = [flux]
    ind_flux['es1_highz_alma_err'] = [err]
    ind_flux.to_csv(bcg+"/flux.csv", index = None)

In [84]:
ind_flux = final_flux_table[final_flux_table['id'] == 'ES1_z_1.40'].copy()
ind_flux["es1_highz_alma"] = [0.465]
ind_flux['es1_highz_alma_err'] = [0.058]
es1z140 = ind_flux[['id', 'redshift', 'des_r', 'des_i', 'des_z', 'des_Y', 'des_r_err', 'des_i_err', 'des_z_err', 'des_Y_err',
       'spitzer.irac.ch1', 'spitzer.irac.ch1_err', 'spitzer.irac.ch2',
       'spitzer.irac.ch2_err', 'spitzer.irac.ch3', 'spitzer.irac.ch3_err',
       'spitzer.irac.ch4', 'spitzer.irac.ch4_err', 'spitzer.mips.24',
       'spitzer.mips.24_err', 'herschel.spire.PSW', 'herschel.spire.PSW_err',
       'herschel.spire.PMW', 'herschel.spire.PMW_err', 'herschel.spire.PLW',
       'herschel.spire.PLW_err', 'es1_highz_alma', 'es1_highz_alma_err']].copy()
es1z140.to_csv("ES1_z_1.40/flux.csv", index = None)

In [34]:
for ind,row in bcg_coords.iterrows():
    with open(row['id'] + "/" + row['id'] + "_bcg.reg","w") as f:
        region = "# Region file format: DS9 version 4.0 \nglobal color=blue font='helvetica 10 normal roman' edit=1 move=1 delete=1 highlite=1 include=1 wcs=wcs \nfk5; cross point " + str(row['ra']) + " " + str(row['dec'])
        f.write(region)

In [85]:
final_flux_table

,id,redshift,des_g,des_r,des_i,des_z,des_Y,des_g_err,des_r_err,des_i_err,des_z_err,des_Y_err,spitzer.irac.ch1,spitzer.irac.ch1_err,spitzer.irac.ch2,spitzer.irac.ch2_err,spitzer.irac.ch3,spitzer.irac.ch3_err,spitzer.irac.ch4,spitzer.irac.ch4_err,spitzer.mips.24,spitzer.mips.24_err,herschel.spire.PSW,herschel.spire.PSW_err,herschel.spire.PMW,herschel.spire.PMW_err,herschel.spire.PLW,herschel.spire.PLW_err
0,ES1_z_0.88,0.563950,4.139844e-03,0.015540,0.026555,0.036578,0.041784,1.451179e-04,0.000200,0.000345,0.000642,0.001631,0.10041,0.00140,0.07239,0.00172,0.08025,0.00665,0.07769,0.00664,0.65296,0.01968,30.12666,4.87164,7.174814,7.983726,6.815913,5.413804
1,ES1_z_0.99,1.193720,1.039369e-03,0.002651,0.004367,0.008354,0.012168,1.266391e-04,0.000173,0.000305,0.000561,0.001577,0.06367,0.00118,0.06687,0.00171,0.00000,0.00000,0.03182,0.00430,0.54488,0.02319,21.52674,4.864613,15.071321,4.989201,14.802241,5.519685
2,ES1_z_0.99b,0.918860,1.446044e-03,0.002966,0.007725,0.012133,0.012241,1.598704e-04,0.000208,0.000364,0.000687,0.001674,0.07258,0.00115,0.05067,0.00151,0.00000,0.00000,0.00000,0.00000,0.47278,0.02584,18.549965,4.89085,16.033789,6.563047,17.865845,4.746464
3,ES1_z_1.04,1.040000,2.245011e-03,0.005632,0.014620,0.022221,0.023969,1.613787e-04,0.000219,0.000411,0.000778,0.001816,0.18666,0.00128,0.12338,0.00209,0.06953,0.00517,0.04078,0.00535,0.98493,0.02391,42.170063,4.862975,18.31567,6.63837,16.833277,4.679536
4,ES1_z_1.38,1.380000,1.716325e-04,0.001332,0.005083,0.009905,0.011606,1.664763e-04,0.000229,0.000424,0.000817,0.001867,0.09046,0.00126,0.08258,0.00173,0.08159,0.00659,0.07309,0.00621,0.60973,0.02572,32.318867,4.906745,28.011467,4.906565,6.738684,6.659844
5,ES1_z_1.40,1.400000,9.120108e-34,0.001277,0.005051,0.007984,0.007734,9.028907e-32,0.000261,0.000436,0.000807,0.002414,0.08509,0.00126,0.06186,0.00144,0.03315,0.00448,0.00000,0.00000,0.5747,0.0215,40.878433,4.89106,22.46016,6.011165,10.882947,8.582891
6,ES1_z_1.60,1.600000,1.016337e-03,0.002563,0.005644,0.012203,0.013620,1.378952e-04,0.000178,0.000311,0.000661,0.001575,0.11297,0.00120,0.10042,0.00146,0.05632,0.00424,0.06321,0.00538,0.63231,0.02415,39.823303,4.870074,27.258715,4.823761,18.17139,4.706377
7,ES1_z_1.65,1.650000,1.542578e-03,0.004242,0.009047,0.017189,0.022600,2.754413e-04,0.000409,0.000651,0.001375,0.003672,0.16125,0.00179,0.17127,0.00225,0.12586,0.00828,0.09394,0.00698,0.4444,0.025,59.64121,4.880317,34.946644,4.808603,16.285534,4.697502
8,ES1_z_1.70,0.916510,6.124790e-04,0.001567,0.006095,0.008364,0.012273,2.028412e-04,0.000238,0.000435,0.000874,0.002316,0.12963,0.00112,0.09562,0.00158,0.05764,0.00420,0.07155,0.00623,1.02563,0.02621,49.65039,4.835531,41.86841,5.425515,6.156414,7.755236
9,XMM_113,1.600000,1.369127e-03,0.002010,0.004391,0.008729,0.010695,2.178991e-04,0.000315,0.000519,0.001174,0.002874,0.09339,0.00125,0.09832,0.00210,0.12585,0.00807,0.00000,0.00000,0.29466,0.01739,13.267184,5.656016,4.917049,4.575537,0.223893,5.079666


In [44]:
des_catalog_fluxes

,des_g,des_r,des_i,des_z,des_Y,des_g_err,des_r_err,des_i_err,des_z_err,des_Y_err,id
0,4.139844e-03,0.015540,0.026555,0.036578,0.041784,1.451179e-04,0.000200,0.000345,0.000642,0.001631,ES1_z_0.88
1,1.039369e-03,0.002651,0.004367,0.008354,0.012168,1.266391e-04,0.000173,0.000305,0.000561,0.001577,ES1_z_0.99
2,1.446044e-03,0.002966,0.007725,0.012133,0.012241,1.598704e-04,0.000208,0.000364,0.000687,0.001674,ES1_z_0.99b
3,2.245011e-03,0.005632,0.014620,0.022221,0.023969,1.613787e-04,0.000219,0.000411,0.000778,0.001816,ES1_z_1.04
4,1.716325e-04,0.001332,0.005083,0.009905,0.011606,1.664763e-04,0.000229,0.000424,0.000817,0.001867,ES1_z_1.38
5,9.120108e-34,0.001277,0.005051,0.007984,0.007734,9.028907e-32,0.000261,0.000436,0.000807,0.002414,ES1_z_1.40
6,1.016337e-03,0.002563,0.005644,0.012203,0.013620,1.378952e-04,0.000178,0.000311,0.000661,0.001575,ES1_z_1.60
7,1.542578e-03,0.004242,0.009047,0.017189,0.022600,2.754413e-04,0.000409,0.000651,0.001375,0.003672,ES1_z_1.65
8,6.124790e-04,0.001567,0.006095,0.008364,0.012273,2.028412e-04,0.000238,0.000435,0.000874,0.002316,ES1_z_1.70
9,1.369127e-03,0.002010,0.004391,0.008729,0.010695,2.178991e-04,0.000315,0.000519,0.001174,0.002874,XMM_113
